# Import Libraries

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('S30 ETL Assignment.db')

In [3]:
cur = con.cursor()

In [4]:
# check for available tables in the db

tables = cur.execute("SELECT name FROM sqlite_schema WHERE type='table'")
tables.fetchall()

[('sqlite_sequence',), ('sales',), ('orders',), ('items',), ('customers',)]

## Method 1: Python

### Sales Table

In [5]:
query = 'select * from sales'

sales_df = pd.read_sql(query, con)
print(sales_df.shape)

(500, 2)


In [6]:
sales_df.head()

,sales_id,customer_id
0,1,10
1,2,85
2,3,42
3,4,93
4,5,75


### Orders Table

In [7]:
query = 'select * from orders'

orders_df = pd.read_sql(query, con)
print(orders_df.shape)

(1500, 4)


In [8]:
orders_df.head()

,order_id,sales_id,item_id,quantity
0,1,1,1,2.0
1,2,1,2,NaN
2,3,1,3,1.0
3,4,2,1,1.0
4,5,2,2,1.0


### Items Table

In [9]:
query = 'select * from items'

items_df = pd.read_sql(query, con)
print(items_df.shape)

(3, 2)


In [10]:
items_df.head()

,item_id,item_name
0,1,x
1,2,y
2,3,z


### Customers Table

In [11]:
query = 'select * from customers'

customers_df = pd.read_sql(query, con)
print(customers_df.shape)

(100, 2)


In [12]:
customers_df.head()

,customer_id,age
0,1,38
1,2,43
2,3,13
3,4,39
4,5,18


In [13]:
customer_sales_df = sales_df.merge(customers_df, on = ['customer_id'], how = 'left')
print(customer_sales_df.shape)

(500, 3)


In [14]:
customer_sales_df.head()

,sales_id,customer_id,age
0,1,10,76
1,2,85,39
2,3,42,59
3,4,93,50
4,5,75,38


In [15]:
orders_items_df = orders_df.merge(items_df, on = ['item_id'], how = 'left')
print(orders_items_df.shape)

(1500, 5)


In [16]:
orders_items_df.head()

,order_id,sales_id,item_id,quantity,item_name
0,1,1,1,2.0,x
1,2,1,2,NaN,y
2,3,1,3,1.0,z
3,4,2,1,1.0,x
4,5,2,2,1.0,y


In [17]:
final_df = orders_items_df.merge(customer_sales_df, on = ['sales_id'], how = 'left')
print(final_df.shape)

(1500, 7)


In [18]:
final_df.head()

,order_id,sales_id,item_id,quantity,item_name,customer_id,age
0,1,1,1,2.0,x,10,76
1,2,1,2,NaN,y,10,76
2,3,1,3,1.0,z,10,76
3,4,2,1,1.0,x,85,39
4,5,2,2,1.0,y,85,39


In [19]:
# check for null values

final_df.isnull().sum()

order_id         0
sales_id         0
item_id          0
quantity       553
item_name        0
customer_id      0
age              0
dtype: int64

In [20]:
# check for data types

final_df.dtypes

order_id         int64
sales_id         int64
item_id          int64
quantity       float64
item_name       object
customer_id      int64
age              int64
dtype: object

In [21]:
final_df['quantity'].unique()

array([ 2., nan,  1.,  3.,  4.,  8.,  7.,  5.])

### Final Filtering

In [22]:
final_df = final_df[(~final_df['quantity'].isnull()) & (final_df['age'] >=18) & (final_df['age'] <=35)]

In [23]:
final_df['quantity'] = final_df['quantity'].astype(int)

In [24]:
final_df.dtypes

order_id        int64
sales_id        int64
item_id         int64
quantity        int64
item_name      object
customer_id     int64
age             int64
dtype: object

In [25]:
final_df.isnull().sum()

order_id       0
sales_id       0
item_id        0
quantity       0
item_name      0
customer_id    0
age            0
dtype: int64

In [26]:
# verifying age value following the conditions

sorted(final_df['age'].unique())

[18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 32, 33, 35]

### Aggregation

In [27]:
final_df = final_df.groupby(['customer_id', 'age', 'item_name'])['quantity'].sum().reset_index()

In [28]:
final_df.sort_values(by = ['customer_id', 'item_name'], inplace = True)

In [29]:
final_df.head()

,customer_id,age,item_name,quantity
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19


In [30]:
final_df.to_csv('python_final.csv', sep = ';', index = False)

## Method 2: SQL

In [31]:
query = """
        select c.customer_id,
                c.age,
                i.item_name,
                sum(cast(o.quantity as int)) as quantity
        from orders o
        left join items i on i.item_id = o.item_id
        left join sales s on s.sales_id = o.sales_id
        left join customers c on c.customer_id = s.customer_id
        where 1=1
        and c.age >= 18 and c.age <= 35
        and quantity is not null
        group by c.customer_id,
                c.age,
                i.item_name
        order by c.customer_id, i.item_name

"""

final_df = pd.read_sql(query, con)
print(final_df.shape)

(70, 4)


In [32]:
final_df.head()

,customer_id,age,item_name,quantity
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19


In [33]:
final_df.to_csv('sql_final.csv', sep = ';', index = False)

In [34]:
con.close()